# Práctica seguimiento

Participantes:
- Gerardo León Quintana
- Susana Suárez Mendoza

In [9]:
import os
import json
import torch
import ast

## 1. Cargar los resultados

In [ ]:
def parse_tensor_list(tensor_list):
    return [t.item() if isinstance(t, torch.Tensor) else t for t in tensor_list]

def parse_txt_file(file_path):
    data = {}
    with open(file_path, 'r') as f:
        for line in f:
            if ':' in line:
                key, value = line.split(':', 1)
                key = key.strip()

                value = ast.literal_eval(value.strip())
                data[key] = parse_tensor_list(value)
    return data

def load_data_from_folder(folder_path):
    data = {}

    for filename in os.listdir(folder_path):

        # abre el contenido del archivo:
        content = open(os.path.join(folder_path, filename)).read()
        content = content.split('\n')
    return data

In [17]:
folder_path = './resultados'

loaded_data = load_data_from_folder(folder_path)
for method, values in loaded_data.items():
    print(f"Método: {method}")
    print(f"Claves disponibles: {list(values.keys())}")
    print(f"Ejemplo de 'x_coordinates': {values['x_coordinates'][:5]}")
    print()

['x_coordinates: [tensor(821.9163), tensor(821.8753), tensor(821.6561), tensor(821.1349), tensor(820.3383), tensor(819.3727), tensor(816.7519), tensor(816.7515), tensor(816.2067), tensor(813.9758), tensor(811.8250), tensor(809.3544), tensor(807.8257), tensor(805.9581), tensor(804.1265), tensor(802.6164), tensor(799.5385), tensor(798.0531), tensor(796.5476), tensor(795.3462), tensor(794.4599), tensor(793.8427), tensor(793.6301), tensor(793.2646), tensor(792.9738), tensor(792.8406), tensor(792.3277), tensor(791.8816), tensor(791.3204), tensor(790.5585), tensor(789.5604), tensor(786.9213), tensor(785.6803), tensor(784.8804), tensor(784.1808), tensor(783.0742), tensor(780.1049), tensor(779.1148), tensor(777.8998), tensor(776.5035), tensor(775.0496), tensor(772.1855), tensor(770.7310), tensor(769.2962), tensor(768.1281), tensor(767.1199), tensor(765.5686), tensor(765.0649), tensor(764.6648), tensor(764.4727), tensor(764.2341), tensor(763.7072), tensor(763.5129), tensor(763.2193), tensor(762

ValueError: malformed node or string on line 1: <ast.Call object at 0x00000176D0FEF910>